<a href="https://colab.research.google.com/github/keejunwei/pi-builder/blob/main/Reinforcement_Learning_in_Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The intent of the work is to first explore how environments are set up for agents to play simple games, before investigating how reinforcement is applied.

We will then try to explore designing environments for other simple game rulesets and highlight key concepts in understanding what reinforcement learning is, before crafting a lesson plan and slides to teach a 2-hour lesson on reinforcement learning, anchoring the lesson in simple understandable games.

In [ ]:
# prompt: Generate two self-running agents playing tic-tac-toe with each other

import random

class TicTacToe: #class serves as a blueprint/ template for creating objects
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # Represents the 3x3 board
        self.current_winner = None # Keeps track of the winner

    def print_board(self):
        # Prints each row of the board
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        # Gives a visual representation of the board with numbers for easy reference
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        # Returns a list of available spots (indices) on the board
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        # Returns True if there are any empty squares, False otherwise
        return ' ' in self.board

    def num_empty_squares(self):
        # Returns the number of empty squares
        return self.board.count(' ')

    def make_move(self, square, letter):
        # Makes a move on the board at the given square with the given letter
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        # Checks if the player with 'letter' has won
        # Check row
        row_ind = square // 3
        row = self.board[row_ind*3 : (row_ind+1)*3]
        if all([spot == letter for spot in row]):
            return True

        # Check column
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        # Check diagonals
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]] # Left to right diagonal
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]] # Right to left diagonal
            if all([spot == letter for spot in diagonal2]):
                return True

        return False # No winner yet

class HumanPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8):')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

class RandomComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        # Gets a random valid available spot for the next move
        square = random.choice(game.available_moves())
        return square

def play(game, x_player, o_player, print_game=True):
    if print_game:
        game.print_board_nums()

    letter = 'X' # Starting letter
    while game.empty_squares():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)

        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                game.print_board()
                print('') # Empty line

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter # Returns the winner
            letter = 'O' if letter == 'X' else 'X' # Switch turns

    if print_game:
        print('It\'s a tie!')

if __name__ == '__main__':
    # Create two random computer players
    x_player = RandomComputerPlayer('X')
    o_player = RandomComputerPlayer('O')
    t = TicTacToe() # Create a new game
    play(t, x_player, o_player, print_game=True) # Play the game with printing enabled


| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
X makes a move to square 4
|   |   |   |
|   | X |   |
|   |   |   |

O makes a move to square 1
|   | O |   |
|   | X |   |
|   |   |   |

X makes a move to square 7
|   | O |   |
|   | X |   |
|   | X |   |

O makes a move to square 8
|   | O |   |
|   | X |   |
|   | X | O |

X makes a move to square 6
|   | O |   |
|   | X |   |
| X | X | O |

O makes a move to square 0
| O | O |   |
|   | X |   |
| X | X | O |

X makes a move to square 3
| O | O |   |
| X | X |   |
| X | X | O |

O makes a move to square 5
| O | O |   |
| X | X | O |
| X | X | O |

X makes a move to square 2
| O | O | X |
| X | X | O |
| X | X | O |

X wins!


In [ ]:
# prompt: Introduce reinforcement learning for the two agents so that they can refine their strategy

import random

class QLearningPlayer:
    def __init__(self, letter, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.letter = letter
        self.q_table = {}  # Q-table to store state-action values
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration-exploitation trade-off
        self.last_state = None
        self.last_action = None

    def get_state(self, game):
        # Convert the board state to a hashable string
        return str(game.board)

    def get_move(self, game):
        state = self.get_state(game)
        available_moves = game.available_moves()

        if not available_moves:
            return None # No moves possible

        # Exploration: Choose a random move with probability epsilon
        if random.uniform(0, 1) < self.epsilon:
            square = random.choice(available_moves)
        # Exploitation: Choose the move with the highest Q-value
        else:
            q_values = {move: self.q_table.get((state, move), 0) for move in available_moves}
            max_q = max(q_values.values())
            # Handle multiple moves with the same maximum Q-value
            best_moves = [move for move, q_value in q_values.items() if q_value == max_q]
            square = random.choice(best_moves)

        self.last_state = state
        self.last_action = square
        return square

    def learn(self, old_state, action, reward, new_state, game):
        # Update Q-value using the Q-learning formula
        old_q_value = self.q_table.get((old_state, action), 0)
        future_q_values = [self.q_table.get((new_state, move), 0) for move in game.available_moves()]
        max_future_q = max(future_q_values) if future_q_values else 0
        new_q_value = old_q_value + self.alpha * (reward + self.gamma * max_future_q - old_q_value)
        self.q_table[(old_state, action)] = new_q_value

def play_with_learning(game, x_player, o_player, print_game=False):
    if print_game:
        game.print_board_nums()

    letter = 'X'
    while game.empty_squares():
        current_player = x_player if letter == 'X' else o_player
        opponent_player = o_player if letter == 'X' else x_player

        old_state = current_player.get_state(game)
        square = current_player.get_move(game)

        if square is None: # Handle case where no moves are available (shouldn't happen in a valid game state)
            break

        if game.make_move(square, letter):
            new_state = current_player.get_state(game)

            # Determine reward
            reward = 0
            if game.current_winner:
                if game.current_winner == current_player.letter:
                    reward = 1 # Win
                else:
                    reward = -1 # Loss
            elif not game.empty_squares():
                reward = 0.5 # Tie

            # Learn from the move
            current_player.learn(old_state, square, reward, new_state, game)

            if game.current_winner:
                # Opponent also needs to learn from this outcome
                if isinstance(opponent_player, QLearningPlayer): #isinstance() function is used to determine if the object is an instance of the given class or any of its subclasses
                     # Opponent gets the opposite reward
                    opponent_player.learn(opponent_player.last_state, opponent_player.last_action, -reward, new_state, game)

                if print_game:
                    print(letter + f' wins!')
                return game.current_winner

            letter = 'O' if letter == 'X' else 'X' # Switch turns
        else:
            # If make_move returns False, it means the chosen square was not available.
            # This should not happen with a QLearningPlayer choosing from available_moves,
            # but as a safeguard, we can give a negative reward and potentially end the game for this player.
            if isinstance(current_player, QLearningPlayer):
                current_player.learn(old_state, square, -1, current_player.get_state(game), game)
            if print_game:
                print(f"{letter} chose an invalid move at square {square}. Ending game.")
            return 'Invalid Move'


    if print_game:
        print('It\'s a tie!')
    # Learn from a tie
    if isinstance(x_player, QLearningPlayer):
        x_player.learn(x_player.last_state, x_player.last_action, 0.5, x_player.get_state(game), game)
    if isinstance(o_player, QLearningPlayer):
        o_player.learn(o_player.last_state, o_player.last_action, 0.5, o_player.get_state(game), game)

    return 'Tie'

if __name__ == '__main__':
    # Create two Q-learning players
    q_player_x = QLearningPlayer('X')
    q_player_o = QLearningPlayer('O')

    num_training_games = 10000

    print(f"Training for {num_training_games} games...")
    for i in range(num_training_games):
        t = TicTacToe()
        play_with_learning(t, q_player_x, q_player_o, print_game=False)

    print("Training complete. Playing a game with learned strategies:")
    t = TicTacToe()
    play_with_learning(t, q_player_x, q_player_o, print_game=True)

    # You can inspect the learned Q-tables (they can be quite large)
    # print("\nX Player Q-table size:", len(q_player_x.q_table))
    # print("O Player Q-table size:", len(q_player_o.q_table))

    # Example of playing against a human with a learned agent
    # print("\nPlaying against a human:")
    # human_player = HumanPlayer('O')
    # t = TicTacToe()
    # play(t, q_player_x, human_player, print_game=True)


Training for 10000 games...
Training complete. Playing a game with learned strategies:
| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
It's a tie!


In [ ]:
import random

# ------------------------------
# Connect Four Game Environment
# ------------------------------

class ConnectFour:
    def __init__(self):
        self.board = [[' ' for _ in range(7)] for _ in range(6)]  # 6 rows x 7 cols
        self.current_winner = None

    def print_board(self):
        for row in self.board:
            print('| ' + ' | '.join(row) + ' |')
        print('-' * 29)
        print(' 0   1   2   3   4   5   6')

    def available_moves(self):
        return [c for c in range(7) if self.board[0][c] == ' ']

    def make_move(self, col, letter):
        for row in reversed(range(6)):
            if self.board[row][col] == ' ':
                self.board[row][col] = letter
                if self.check_winner(row, col, letter):
                    self.current_winner = letter
                return True
        return False

    def check_winner(self, row, col, letter):
        def count_direction(r_step, c_step):
            count = 0
            r, c = row, col
            while 0 <= r < 6 and 0 <= c < 7 and self.board[r][c] == letter:
                count += 1
                r += r_step
                c += c_step
            return count - 1

        directions = [(0, 1), (1, 0), (1, 1), (1, -1)]
        for dr, dc in directions:
            total = 1 + count_direction(dr, dc) + count_direction(-dr, -dc)
            if total >= 4:
                return True
        return False

    def empty_squares(self):
        return any(' ' in row for row in self.board)

    def num_empty_squares(self):
        return sum(row.count(' ') for row in self.board)

# ------------------------------
# Q-Learning Player Class
# ------------------------------

class QLearningPlayer:
    def __init__(self, letter, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.letter = letter
        self.q_table = {}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.last_state = None
        self.last_action = None

    def get_state(self, game):
        return ''.join([''.join(row) for row in game.board])

    def get_move(self, game):
        state = self.get_state(game)
        available_moves = game.available_moves()

        if not available_moves:
            return None

        if random.uniform(0, 1) < self.epsilon:
            square = random.choice(available_moves)
        else:
            q_values = {move: self.q_table.get((state, move), 0) for move in available_moves}
            max_q = max(q_values.values())
            best_moves = [move for move, q in q_values.items() if q == max_q]
            square = random.choice(best_moves)

        self.last_state = state
        self.last_action = square
        return square

    def learn(self, old_state, action, reward, new_state, game):
        old_q_value = self.q_table.get((old_state, action), 0)
        future_q_values = [self.q_table.get((new_state, move), 0) for move in game.available_moves()]
        max_future_q = max(future_q_values) if future_q_values else 0
        new_q_value = old_q_value + self.alpha * (reward + self.gamma * max_future_q - old_q_value)
        self.q_table[(old_state, action)] = new_q_value

# ------------------------------
# Training & Playing Function
# ------------------------------

def play_with_learning(game, x_player, o_player, print_game=False):
    if print_game:
        print("Initial Board:")
        game.print_board()

    letter = 'X'
    while game.empty_squares():
        current_player = x_player if letter == 'X' else o_player
        opponent_player = o_player if letter == 'X' else x_player

        old_state = current_player.get_state(game)
        move = current_player.get_move(game)

        if move is None:
            break

        if game.make_move(move, letter):
            new_state = current_player.get_state(game)

            reward = 0
            if game.current_winner:
                if game.current_winner == current_player.letter:
                    reward = 1
                else:
                    reward = -1
            elif not game.empty_squares():
                reward = 0.5

            current_player.learn(old_state, move, reward, new_state, game)

            if print_game:
                print(f"{letter} moves in column {move}:")
                game.print_board()
                print()

            if game.current_winner:
                if isinstance(opponent_player, QLearningPlayer):
                    opponent_player.learn(
                        opponent_player.last_state,
                        opponent_player.last_action,
                        -reward,
                        new_state,
                        game
                    )
                if print_game:
                    print(letter + " wins!")
                return game.current_winner

            letter = 'O' if letter == 'X' else 'X'
        else:
            if isinstance(current_player, QLearningPlayer):
                current_player.learn(old_state, move, -1, current_player.get_state(game), game)
            if print_game:
                print(f"{letter} chose an invalid move at column {move}. Ending game.")
            return 'Invalid Move'

    if print_game:
        print("It's a tie!")
        game.print_board()

    if isinstance(x_player, QLearningPlayer):
        x_player.learn(x_player.last_state, x_player.last_action, 0.5, x_player.get_state(game), game)
    if isinstance(o_player, QLearningPlayer):
        o_player.learn(o_player.last_state, o_player.last_action, 0.5, o_player.get_state(game), game)

    return 'Tie'

# ------------------------------
# Main Training Loop
# ------------------------------

if __name__ == '__main__':
    q_player_x = QLearningPlayer('X')
    q_player_o = QLearningPlayer('O')

    num_training_games = 5000

    print(f"Training Connect Four for {num_training_games} games...")
    for i in range(num_training_games):
        game = ConnectFour()
        play_with_learning(game, q_player_x, q_player_o, print_game=False)

        if i % 500 == 0:
            print(f"Completed {i} games")

    print("Training complete. Playing one demonstration game:\n")
    game = ConnectFour()
    play_with_learning(game, q_player_x, q_player_o, print_game=True)


Training Connect Four for 5000 games...
Completed 0 games
Completed 500 games
Completed 1000 games
Completed 1500 games
Completed 2000 games
Completed 2500 games
Completed 3000 games
Completed 3500 games
Completed 4000 games
Completed 4500 games
Training complete. Playing one demonstration game:

Initial Board:
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
-----------------------------
 0   1   2   3   4   5   6
X moves in column 1:
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   | X |   |   |   |   |   |
-----------------------------
 0   1   2   3   4   5   6

O moves in column 5:
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |
|   | X |   |   |   

In [ ]:
import random
from collections import defaultdict

NUM_COLOURS = 5
CODE_LENGTH = 3
ALL_GUESSES = [[i, j, k] for i in range(NUM_COLOURS) for j in range(NUM_COLOURS) for k in range(NUM_COLOURS)]

def generate_secret_code():
    return [random.randint(0, NUM_COLOURS - 1) for _ in range(CODE_LENGTH)]

def get_feedback(secret, guess):
    feedback = 0
    used_secret = [False] * CODE_LENGTH
    used_guess = [False] * CODE_LENGTH

    # +2: correct colour and position
    for i in range(CODE_LENGTH):
        if guess[i] == secret[i]:
            feedback += 2
            used_secret[i] = True
            used_guess[i] = True

    # +1: correct colour, wrong position
    for i in range(CODE_LENGTH):
        if not used_guess[i]:
            for j in range(CODE_LENGTH):
                if not used_secret[j] and guess[i] == secret[j]:
                    feedback += 1
                    used_secret[j] = True
                    used_guess[i] = True
                    break

    # -1: wrong colour
    wrongs = CODE_LENGTH - sum(used_guess)
    feedback -= wrongs
    return feedback

class SmartQLearningAgent:
    def __init__(self, alpha=0.3, gamma=0.9, epsilon=0.1):
        self.q_table = defaultdict(float)
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def get_state(self, known_correct, known_wrong, last_feedback):
        return (tuple(sorted(known_correct)), tuple(sorted(known_wrong)), last_feedback)

    def choose_action(self, state, tried, known_wrong):
        valid_colours = [c for c in range(NUM_COLOURS) if c not in known_wrong]
        possible_actions = [
            g for g in ALL_GUESSES
            if all(c in valid_colours for c in g) and tuple(g) not in tried
        ]

        if not possible_actions:
            return random.choice(ALL_GUESSES)

        if random.random() < self.epsilon:
            return random.choice(possible_actions)

        q_values = {tuple(a): self.q_table[(state, tuple(a))] for a in possible_actions}
        max_q = max(q_values.values())
        best_actions = [list(a) for a, q in q_values.items() if q == max_q]
        return random.choice(best_actions)

    def update_q(self, state, action, reward, next_state):
        action = tuple(action)
        future_qs = [self.q_table[(next_state, tuple(a))] for a in ALL_GUESSES]
        max_future_q = max(future_qs)
        current_q = self.q_table[(state, action)]
        self.q_table[(state, action)] = current_q + self.alpha * (reward + self.gamma * max_future_q - current_q)

def train(agent, episodes=20000):
    for ep in range(episodes):
        secret = generate_secret_code()
        tried = set()
        known_wrong = set()
        known_correct = set()
        last_feedback = 0

        for turn in range(17):
            state = agent.get_state(known_correct, known_wrong, last_feedback)
            guess = agent.choose_action(state, tried, known_wrong)
            tried.add(tuple(guess))

            feedback = get_feedback(secret, guess)

            if feedback == -3:
                known_wrong.update(guess)
            elif feedback >= 3:
                for c in guess:
                    known_correct.add(c)

            next_state = agent.get_state(known_correct, known_wrong, feedback)
            agent.update_q(state, guess, feedback, next_state)

            last_feedback = feedback
            if feedback == 6:
                break

def play(agent):
    secret = generate_secret_code()
    print("Secret Code (Hidden):", secret)
    tried = set()
    known_wrong = set()
    known_correct = set()
    last_feedback = 0

    for turn in range(17):
        state = agent.get_state(known_correct, known_wrong, last_feedback)
        guess = agent.choose_action(state, tried, known_wrong)
        tried.add(tuple(guess))

        feedback = get_feedback(secret, guess)
        print(f"Turn {turn+1}: Guess = {guess}, Feedback = {feedback}")

        if feedback == -3:
            known_wrong.update(guess)
        elif feedback >= 3:
            for c in guess:
                known_correct.add(c)

        last_feedback = feedback
        if feedback == 6:
            print("✅ Code cracked!")
            return

    print("❌ Failed to crack the code.")

if __name__ == "__main__":
    agent = SmartQLearningAgent()
    print("Training agent...")
    train(agent, episodes=20000)
    print("Training complete.\nNow playing:")
    play(agent)

Training agent...
Training complete.
Now playing:
Secret Code (Hidden): [2, 3, 0]
Turn 1: Guess = [3, 0, 2], Feedback = 3
Turn 2: Guess = [0, 3, 1], Feedback = 2
Turn 3: Guess = [0, 2, 4], Feedback = 1
Turn 4: Guess = [0, 3, 4], Feedback = 2
Turn 5: Guess = [0, 3, 0], Feedback = 3
Turn 6: Guess = [0, 4, 2], Feedback = 1
Turn 7: Guess = [4, 3, 2], Feedback = 2
Turn 8: Guess = [1, 3, 0], Feedback = 3
Turn 9: Guess = [1, 0, 3], Feedback = 1
Turn 10: Guess = [0, 4, 3], Feedback = 1
Turn 11: Guess = [3, 4, 0], Feedback = 2
Turn 12: Guess = [2, 0, 3], Feedback = 4
Turn 13: Guess = [0, 0, 2], Feedback = 1
Turn 14: Guess = [3, 2, 1], Feedback = 1
Turn 15: Guess = [2, 1, 3], Feedback = 2
Turn 16: Guess = [2, 3, 0], Feedback = 6
✅ Code cracked!
